# TESLA 9-cell Cavity



In [ ]:
from impact import Impact
from pmd_beamphysics.units import mec2
import numpy as np
import os

import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

In [ ]:
ifile1 = "../templates/tesla_9cell_cavity/ImpactT-1d.in"
ifile2 = "../templates/tesla_9cell_cavity/ImpactT-2d.in"
os.path.exists(ifile1), os.path.exists(ifile2)

In [ ]:
# calculate gamma*beta
Etot = 10e6  # eV
gamma = Etot / mec2
GB = np.sqrt(gamma**2 - 1)
GB

GB, np.sqrt(Etot**2 - mec2**2)

In [ ]:
%%time
I1 = Impact(ifile1)
I1.run()
I1.plot(y2="mean_kinetic_energy")

In [ ]:
%%time
I2 = Impact(ifile2)
I2.run()
I2.plot(y2="mean_kinetic_energy")

In [ ]:
I1.particles["final_particles"].plot("z", "pz", bins=100)

In [ ]:
I2.particles["final_particles"].plot("z", "pz", bins=100)

In [ ]:
# ky = 'mean_kinetic_energy'
def compare(ky="sigma_x", kx="mean_z"):
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(I1.stat(kx), I1.stat(ky), label="Impact-T 1D")
    ax.plot(I2.stat(kx), I2.stat(ky), label="Impact-T 2D", linestyle="--")
    ax.legend()
    # ax.set_xlim(0,2)

    ax.set_xlabel(kx)
    ax.set_ylabel(ky)


compare()

# Use Impact's built-in Gaussian particle generator

In [ ]:
I = Impact(ifile1)

I.header["Dt"] = 1e-12
I.total_charge = 0  # Turn SC off
print(I)
I.run()
I.particles["final_particles"]["sigma_x"] / 1e-6

In [ ]:
I.particles["final_particles"].plot("delta_z", "delta_pz")

In [ ]:
I.lattice

# Autophase

In [ ]:
from impact.autophase import autophase

from pmd_beamphysics import single_particle

P0 = single_particle(pz=10e6, z=1e-15)

autophase(
    I2,
    ele_name="cavity9",
    initial_particles=P0,
    metric="mean_kinetic_energy",
    verbose=True,
)

In [ ]:
I.run()

In [ ]:
(
    I.particles["final_particles"]["mean_energy"] / 1e6,
    I.particles["final_particles"]["sigma_x"] / 1e-6,
)

In [ ]:
# Compare these.
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)

In [ ]:
I.output["stats"].keys()

In [ ]:
PI = I.particles["initial_particles"]
PF = I.particles["final_particles"]

In [ ]:
PI["mean_pz"]

In [ ]:
PF.plot("delta_z", "delta_pz")

# Make particles in distgen

In [ ]:
from distgen import Generator

YAML = """
n_particle: 10000
random_type: hammersley
species: electron
start:
  tstart:
    units: sec
    value: 0
  type: time
total_charge:
  units: nC
  value: 1
r_dist:
  sigma_xy:
    units: mm
    value: .01
  type: radial_gaussian
z_dist:
  avg_z:
    units: mm
    value: 0
  sigma_z:
    units: mm
    value: 0.1
  type: gaussian
  

transforms:
  setPz:
    type: set_avg pz
    avg_pz: 
      value: 99.99869439159244
      units: MeV/c
  
"""
G = Generator(YAML)
G.run()
P = G.particles

In [ ]:
I2 = I.copy()
I2.initial_particles = P
I2.configure()
I2.total_charge = 0  # or: P['charge']
I2.run()
PF2 = I2.particles["final_particles"]

In [ ]:
PF2.plot("x", "px")
PF2.plot("delta_z", "delta_pz")

# Compare

In [ ]:
# for k in ['x', 'px', 'z', 'pz']:
#    plt.hist(PF[k], density=True, bins=100, label='Impact-T generator', alpha=0.5)
#    plt.hist(PF2[k], density=True, bins=100, label='Distgen generator', alpha=0.5)
#    plt.xlabel(k)
#    plt.legend()
#    plt.show()